# All necessary imports

In [1]:
import sys
sys.path.append('..')

In [2]:
from source.code.utils import save_obj
from source.code.utils import load_obj

In [3]:
import pandas as pd
import numpy as np
import os
import pandas_profiling
from tqdm import tqdm

In [4]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 30000)
pd.set_option('display.max_columns', 30000)

In [5]:
data_path = '../data/dataset/original/{}.csv'
profiling_path = '../data/data_profiling/{}.html'
meta_path = '../data/meta-info/{}.pkl'

In [71]:
QUERY_PATTERN = 'n_missing <= 0 & type == \'{}\''

# Data reading

Firstly we just load all data into memory, then profile each dataset,

then try to filter features that are most interesting for us at the moment

(continuous, categorical, binary features without na, features with low na percentage etc.).

In [6]:
dataset_names = ['application_train', 'application_test', 'bureau', 'bureau_balance', 'credit_card_balance', 'installments_payments', 'POS_CASH_balance', 'previous_application', 'sample_submission']

In [7]:
data_dict = dict(zip(dataset_names, list(map(lambda name: pd.read_csv(filepath_or_buffer=data_path.format(name)), tqdm(dataset_names)))))

100%|██████████| 9/9 [00:47<00:00,  5.26s/it]


In [8]:
columns_description = pd.read_csv(filepath_or_buffer='../data/dataset/HomeCredit_columns_description.csv', encoding='ISO-8859-1', index_col=0)

# Feature description

Here at this picture the general data structure is reflected.

Lots of connections and, as a consequence, lots of hypothetial issues with data.

![Image of data scheme](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

# General data description

- **application_{train|test}.csv**

This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
Static data for all applications. One row represents one loan in our data sample.

- **bureau.csv**

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

- **bureau_balance.csv**

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

- **POS_CASH_balance.csv**

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

- **credit_card_balance.csv**

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

- **previous_application.csv**

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.

- **installments_payments.csv**

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

- **HomeCredit_columns_description.csv**

This file contains descriptions for the columns in the various data files.

# Pandas profiling

Usually this reports are being displayed in notebooks but since train and test have so many variables

it is more convnient to work with report as a Pandas DataFrame which contains meta-information about dataset columns

(number of continuous, categorical, binary columns, number of highly correlated columns etc.).

In [9]:
profiles_dict = {k: pandas_profiling.ProfileReport(v) for k, v in tqdm(data_dict.items())}

100%|██████████| 9/9 [15:41<00:00, 104.57s/it]


# Store the output

We will store all these reports in case.

Also it would be reasonable to store all meta-information to be able not to recalculate it each time.

# Store visual (html) reports

In [10]:
_ = list(map(lambda name: profiles_dict[name].to_file(outputfile=profiling_path.format(name)), tqdm(dataset_names)))

100%|██████████| 9/9 [00:00<00:00, 323.73it/s]


# Store reports with feature meta-information

In [11]:
_ = list(map(lambda name: save_obj(profiles_dict[name].description_set['variables'], meta_path.format(name)), tqdm(dataset_names)))

100%|██████████| 9/9 [00:00<00:00, 652.11it/s]


# Explore variables

Here we just take a look at keys of profiling report dictionary for train (yes, dictionaries, dictionaries and, once again, dictionaries)))

In [12]:
profiles_dict[dataset_names[0]].description_set.keys()

dict_keys(['table', 'variables', 'freq', 'correlations'])

In [13]:
pd.DataFrame(data=profiles_dict[dataset_names[0]].description_set['variables'].columns.values, columns=['PROFILING_VARIABLES'])

,PROFILING_VARIABLES
0,type
1,correlation_var
2,correlation
3,count
4,distinct_count
5,is_unique
6,memorysize
7,mode
8,n_infinite
9,n_missing


As you can see the report contains a value with the 'variables' key.

The value is a Pandas DataFrame which contains informations about each column of the dataset.

Each column is described with features displayed above.

Let's take a closer look.

# TRAIN

In [14]:
name = 0

In [15]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

NUM     39
CORR    34
BOOL    33
CAT     16
Name: type, dtype: int64

In [16]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

88

In [17]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

88

In [18]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

0.000000    54
0.135016    6 
0.003320    3 
0.487810    1 
0.683862    1 
0.507497    1 
0.585160    1 
0.698723    1 
0.000003    1 
0.563811    1 
0.503488    1 
0.002146    1 
0.198253    1 
0.678486    1 
0.501761    1 
0.313455    1 
0.593767    1 
0.694330    1 
0.551792    1 
0.659908    1 
0.508408    1 
0.664978    1 
0.000007    1 
0.532960    1 
0.473983    1 
0.497608    1 
0.004201    1 
0.000039    1 
Name: p_missing, dtype: int64

In [19]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

122

You have probably guessed that the whole count of columns is the sum of count of columns with 'CORR' type and count of columns with some number of missing values.

# TEST

In [20]:
name = 1

In [21]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

NUM      39
CORR     34
BOOL     21
CAT      16
CONST    11
Name: type, dtype: int64

In [22]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

87

In [23]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

87

In [24]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

0.000000    55
0.124097    6 
0.000595    3 
0.652757    1 
0.579641    1 
0.490050    1 
0.687161    1 
0.516761    1 
0.455625    1 
0.483731    1 
0.421221    1 
0.000164    1 
0.666051    1 
0.018689    1 
0.662892    1 
0.684125    1 
0.535122    1 
0.468899    1 
0.490173    1 
0.567065    1 
0.177827    1 
0.478438    1 
0.672842    1 
0.484552    1 
0.320142    1 
0.000492    1 
Name: p_missing, dtype: int64

In [25]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

121

Again, this time the whole count of columns is 1 columns less because test dataset does not contain TARGET column.

# BUREAU

In [26]:
name = 2

In [27]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

NUM    14
CAT    3 
Name: type, dtype: int64

In [28]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

17

In [29]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

17

In [30]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

0.000000    10
0.369170    1 
0.061496    1 
0.344774    1 
0.150119    1 
0.000008    1 
0.655133    1 
0.714735    1 
Name: p_missing, dtype: int64

In [31]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

17

# BUREAU BALANCE

In [32]:
name = 3

In [33]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

NUM    2
CAT    1
Name: type, dtype: int64

In [34]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

3

In [35]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

3

In [36]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

0.0    3
Name: p_missing, dtype: int64

In [37]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

3

# CREDIT CARD BALANCE

In [38]:
name = 4

In [39]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

NUM     17
CORR    5 
CAT     1 
Name: type, dtype: int64

In [40]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

18

In [41]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

18

In [42]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

0.000000    10
0.195249    5 
0.079482    2 
0.199981    1 
Name: p_missing, dtype: int64

In [43]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

23

# INSTALLMENTS PAYMENTS

In [44]:
name = 5

In [45]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

NUM     6
CORR    2
Name: type, dtype: int64

In [46]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

6

In [47]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

6

In [48]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

0.0    6
Name: p_missing, dtype: int64

In [49]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

8

# POS CASH BALANCE

In [50]:
name = 6

In [51]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

NUM    7
CAT    1
Name: type, dtype: int64

In [52]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

8

In [53]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

8

In [54]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

0.000000    6
0.002608    1
0.002607    1
Name: p_missing, dtype: int64

In [55]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

8

# PREVIOUS APPLICATIONS

In [56]:
name = 7

In [57]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

NUM     17
CAT     16
CORR    3 
BOOL    1 
Name: type, dtype: int64

In [58]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

34

In [59]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

34

In [60]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

0.000000    21
0.402981    5 
0.996437    2 
0.536365    2 
0.000207    1 
0.491198    1 
0.222864    1 
0.222867    1 
Name: p_missing, dtype: int64

In [61]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

37

So far we can say that every dataset has some problems with data quality.

It si possble to highlight two major issues at the moment:
- NAs;
- high corelation.

Perhaps (even very likely) there are other issues but that is not clear so far.

Now to simplify the process we will firstly take those features that do not contain any NaN values and are not highly correlated with other features:

In [78]:
datasets_num_features = dict(zip(dataset_names, list(map(lambda name: profiles_dict[name].description_set['variables'].query(QUERY_PATTERN.format('NUM'))['type'].index.values, dataset_names))))

In [79]:
datasets_cat_features = dict(zip(dataset_names, list(map(lambda name: profiles_dict[name].description_set['variables'].query(QUERY_PATTERN.format('CAT'))['type'].index.values, dataset_names))))

In [80]:
datasets_bin_features = dict(zip(dataset_names, list(map(lambda name: profiles_dict[name].description_set['variables'].query(QUERY_PATTERN.format('BOOL'))['type'].index.values, dataset_names))))

In [84]:
save_obj(datasets_num_features, meta_path.format('datasets_num_features'))

In [85]:
save_obj(datasets_cat_features, meta_path.format('datasets_cat_features'))

In [86]:
save_obj(datasets_bin_features, meta_path.format('datasets_bin_features'))